# データドリフトの監視

時間が経つにつれて、モデルは、機能データの変化する傾向により、正確に予測する効果が低下する可能性があります。この現象は「データドリフト」と呼ばれ、必要に応じてモデルを再トレーニングできるように、機械学習ソリューションを監視して検出することが重要です。

このラボでは、データセットのデータドリフトモニタリングを設定します。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## ベースラインデータセットを作成する

データドリフトのデータセットを監視するには、*ベースライン*データセット（通常、モデルのトレーニングに使用されるデータセット）を登録して、将来収集されるデータとの比較ポイントとして使用する必要があります。 

In [ ]:
from azureml.core import Datastore, Dataset


# Upload the baseline data
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-baseline',
                       overwrite=True, 
                       show_progress=True)

# Create and register the baseline dataset
print('Registering baseline dataset...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-baseline/*.csv'))
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='diabetes baseline',
                           description='diabetes baseline data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Baseline dataset registered!')

## ターゲットデータセットを作成する

時間が経つにつれて、ベースライントレーニングデータと同じ機能を備えた新しいデータを収集できます。この新しいデータをベースラインデータと比較するには、データドリフトを分析する機能を含むターゲットデータセットと、新しいデータが最新であった時点を示すタイムスタンプフィールドを定義する必要があります。これにより、時間間隔でのデータドリフトを測定します。タイムスタンプは、データセット自体のフィールドにすることも、データの保存に使用するフォルダーとファイル名のパターンから派生させることもできます。たとえば、月のフォルダーを含む年のフォルダーと、その日のフォルダーを含むフォルダー階層で新しいデータを保存できます。または、次のようにファイル名に年、月、日をエンコードすることもできます。*data_2020-01-29.csv*;これは、次のコードで採用されているアプローチです。

In [ ]:
import datetime as dt
import pandas as pd

print('Generating simulated data...')

# Load the smaller of the two data files
data = pd.read_csv('data/diabetes2.csv')

# We'll generate data for the past 6 weeks
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    # Get the date X weeks ago
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # Modify data to ceate some drift
    data['Pregnancies'] = data['Pregnancies'] + 1
    data['Age'] = round(data['Age'] * 1.2).astype(int)
    data['BMI'] = data['BMI'] * 1.1
    
    # Save the file with the date encoded in the filename
    file_path = 'data/diabetes_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# Upload the files
path_on_datastore = 'diabetes-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# Use the folder partition format to define a dataset with a 'date' timestamp column
partition_format = path_on_datastore + '/diabetes_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# Register the target dataset
print('Registering target dataset...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='diabetes target',
                                                                          description='diabetes target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Target dataset registered!')

## データドリフトモニターを作成する

これで、糖尿病データのデータドリフトモニターを作成する準備が整いました。データドリフトモニターは定期的にまたはオンデマンドで実行され、ベースラインデータセットとターゲットデータセットを比較します。ターゲットデータセットには、新しいデータが徐々に追加されます。

データドリフトを監視する機能、監視プロセスの実行に使用する計算ターゲットの名前、データを比較する頻度、アラートがトリガーされるデータドリフトのしきい値、データの収集を可能にする待ち時間（時間単位）。

In [ ]:
from azureml.datadrift import DataDriftDetector

# set up feature list
features = ['Pregnancies', 'Age', 'BMI']

# set up data drift detector
monitor = DataDriftDetector.create_from_datasets(ws, 'diabetes-drift-detector', baseline_data_set, target_data_set,
                                                      compute_target='aml-cluster', 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

## モニターのバックフィル

6週間のシミュレートされた毎週のデータ収集を含むベースラインデータセットとターゲットデータセットがあります。これを使用してモニターを埋め戻し、元のベースラインとターゲットデータ間のデータドリフトを分析できます。

> **Note** バックフィル分析を実行するには計算ターゲットを起動する必要があるため、これには実行に時間がかかる場合があります。ウィジェットが常に更新されてステータスが表示されるとは限らないため、リンクをクリックして、Azure Machine Learning Studioで実験ステータスを確認してください！

In [ ]:
from azureml.widgets import RunDetails

backfill = monitor.backfill( dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

## データドリフトの分析

次のコードを使用して、バックフィル実行で収集された時点のデータドリフトを調べることができます。

In [ ]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

次の手順に従って、[Azure Machine Learning studio](https://ml.azure.com)でデータドリフトメトリックを視覚化することもできます。

1. [**データセット**]ページで、[**データセットモニター**]タブを表示します。
2. 表示するデータドリフトモニターをクリックします。
3. データドリフトメトリックを表示する日付範囲を選択します（列グラフに複数週間のデータが表示されない場合は、1分ほど待ってから[**更新**]をクリックします）。
4. 上部の**ドリフトの概要**セクションのチャートを調べてください。これは、全体的なドリフトの大きさとフィーチャごとのドリフトの寄与を示しています。
5. 下部の[**機能の詳細**]セクションのグラフをご覧ください。個々の機能のさまざまなドリフトの測定値を確認できます。

> **Note**: データドリフトメトリックの理解については、次の[データセットを監視する方法](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets#understanding-data-drift-results)を参照してください。 Azure Machine Learningのドキュメント。

## さらに探索する

このラボは、データドリフトモニタリングの概念と原理を紹介するように設計されています。データセットを使用したデータドリフトの監視の詳細については、Azure machine Learningドキュメントの[データセットのデータドリフトの検出](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets)を参照してください。 。

Azure Kubernetes Service（AKS）クラスターにデプロイされたサービスのデータドリフトモニタリングを構成することもできます。この詳細については、「Azure Kubernetes Service（AKS）にデプロイされたモデルのデータドリフトを検出する」(https://docs.microsoft.com/azure/machine-learning/how-to-monitor-data-drift)を参照してくださいAzure Machine Learningのドキュメント。
